# Cervical cancer prevention through colposcopy and deep learning

## Preliminary steps

**Before you start using this notebook please enable the free GPU available.** Without it the notebook will not work. For doing so navigate to the menu "Runtime" and select "Change runtime type". Then enable the GPU from the "Hardware accelerator" section. 

If you are not used to Jupyter notebooks or Google Colab please look into this tutorial before starting:
https://colab.research.google.com/notebooks/welcome.ipynb

## What is a colposcopy

The cervix cancer, or cervical cancer is a very dangerous condition which in most of the cases can be avoided through a simple medical inspection named colposcopy. Colposcopy is a relatively non-invasive procedure which consists in the visual inspection of the cervix after applying an acetic acid solution to it. After the application of the solution, the ill areas of the cervix changes colour becoming whiter, enabling the doctor to better asses the state of the patient. The purpose of the colposcopy is not only to detect cancerous tissues, but especially to detect lesions of the cervix which could degenerate into cancer over time. The lesions are usually named CIN(cervical intraepithelial neoplasia) and are split in three classes: CIN1, CIN2, CIN3, based on their severity. The issue with colposcopy is that it requires highly trained professionals in order to be accurate, i.e., even medical personnel can make mistakes when setting a diagnostic based on colposcopy. Researchers are trying to boost the accuracy of colposcopy so that it surpasses human accuracy.

## Dataset description

For this case we propose the cervigram image dataset published on IEEE Dataport. The dataset contains 3339 images, from 477 colposcopy procedures; 3003 images belong to the training set and 336 to the validation set. Each colposcopy procedure is represented in the dataset by seven images, five of them are a sequence displaying the transformation of the cervix after applying acetic acid solution. The last two images are different, one of them show the cervix through a green lens, while the other one shows the cervix after applying iodine solution, a solution of a dark red color. For simplicity, in our case, we will focus only on the images involving acetic acid solution. The dataset is aimed at classification problems, the cervical lesions are divided in four categories: class zero(normal cervix), class one(CIN1), class two(CIN2/3) and class three(cervical cancer). 

## Acquiring the dataset

The dataset is available at:
https://ieee-dataport.org/documents/cervigram-image-dataset

You need to sign-up in order to be able to download the dataset. You can also sign-up as a beta tester and get a free subscription. 

Once you downloaded the dataset you will have a file named "data.zip". The simplest way is to upload this file manually in Google Colab. Check out the left side region of the Colab window for uploading the file, you should look for an icon that looks like a folder. This will open up the "Files" section. In there, you will notice an "Upload" button. Even though it is the most simple way, we **do not recommend** working like this. The upload may be slow and when working on Google Colab you are actually on a virtual machine, which gets wiped out once you disconnect thus, the files will get lost. You would need to do the upload each time you reconnect.

A better way of working would be to upload the dataset to your Google Drive. Then, each time you reconnect, you can mount your Google Drive to the Colab environment. The process is simple and will save a lot of time in the long run. In order to mount your Google Drive, you need to run the code below. You should note the mounted drive in the files section of Colab, please check the left side of the window.

In [0]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
except Exception as e:
  print('Are you running in a Google Colab environment ?')
  print(e)

The script below, clears the dataset folder, then it recreates it. Next, it unzips the dataset into it. In the previous step you either uploaded the file directly from Colab, either to Google Drive, depending on your option please uncomment the relevant line in the script and comment the other. **The path to the file may not match exactly, please modify accordingly.** Once the script finishes please refresh the files view to notice the downloaded files. Inside the dataset folder you should notice a "data" folder containing two sub-folders "train" and "test". Each of those folders contain folders labeled from zero to three, those correspond to each of the classes.

**Please note the way to run UNIX commands from Colab, simply put a "!" in front of the statement.**

In [0]:
!rm -vrf "dataset"
!mkdir "dataset"

# If using Google Drive:
!cp -r "/content/drive/My Drive/data.zip" "dataset/data.zip"

# If not using Google Drive
# !cp -r "/content/data.zip" "dataset/data.zip"

!unzip "dataset/data.zip" -d "dataset"

## Download dependencies

**pip is the default Python package manager**, it gets installed together with Python. The command below installs the dependencies we need.

In [0]:
!pip3 install torch torchvision sklearn matplotlib GPUtil

## Constants

In [0]:
DATASET_PATH = '/content/dataset/data/'
TRAIN_PATH = DATASET_PATH + 'train/'
TEST_PATH = DATASET_PATH + 'test/'
IMAGE_SIZE = 224
BATCH_SIZE = 100
LEARNING_RATE = 1e-4
EPOCHS = 10

## Imports

We import the packages needed.

In [0]:
import torch as t
import torchvision as tv
import numpy as np
import PIL as pil
import matplotlib.pyplot as plt
import sklearn as sk
import sklearn.metrics
import os
import time
import random
import GPUtil

## Memory management

While training neural networks, it's not uncommon for a GPU to run out of memmory. The cell below displays how much memory has each GPU and how much of it is already allocated. If running out of memory you should restart the runtime, if this does not fix the problem please gradually decrease the batch size.

In [0]:
def memory_stats():
  count = 1
  for gpu in GPUtil.getGPUs():
    print('GPU', count, 'Memory')
    print('-----------------------')
    print('Total:', gpu.memoryTotal)
    print('Free:', gpu.memoryFree)
    print('Used:', gpu.memoryUsed)
    print('Used percent:', gpu.memoryUtil * 100, '%')
    print('-----------------------')
    count += 1  

memory_stats()

## Deterministic measurements

This statements help making the experiments reproducible by fixing the random seeds. Despite fixing the random seeds, experiments are usually not reproducible using different PyTorch releases, commits, platforms or between CPU and GPU executions. Please find more details in the PyTorch documentation:

https://pytorch.org/docs/stable/notes/randomness.html

In [0]:
SEED = 0
t.manual_seed(SEED)
t.cuda.manual_seed(SEED)
t.cuda.manual_seed_all(SEED)
t.backends.cudnn.deterministic = True
t.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)

## Loading data

The dataset is structured in multiple small folders of 7 images each. This generator iterates through the folders and returns the category and 7 paths: one for each image in the folder. The paths are ordered; the order is important since each folder contains 3 types of images, first 5 are with acetic acid solution and the last two are through a green lens and having iodine solution(a solution of a dark red color). In our case we will work only with the first 5 images(with acetic acid). Taking advantage of the rest of images is left as an open question/exercise to the reader.

Please note the yield statement in the second function, this turns the function in a Python generator. If you are not aware of what a Python generator is please read this resource:
https://wiki.python.org/moin/Generators 

In [0]:
def sortByLastDigits(elem):
  chars = [c for c in elem if c.isdigit()]
  return 0 if len(chars) == 0 else int(''.join(chars))

def getImagesPaths(root_path):
  for class_folder in [root_path + f for f in os.listdir(root_path)]:
      category = int(class_folder[-1])
      for case_folder in os.listdir(class_folder):
        case_folder_path = class_folder + '/' + case_folder + '/'
        img_files = [case_folder_path + file_name for file_name in os.listdir(case_folder_path)]
        yield category, sorted(img_files, key = sortByLastDigits)

We define a dataset which can apply dynamic and static transformations to images. The static transformations are applied on the initialization of the dataset, while the dynamic ones are applied every time when loading a batch of images.

Defining a dataset requires us to extend a PyTorch class "torch.utils.data.Dataset" and overwrite the methods __getitem__ and __len__. For more details please view this interesting tutorial from the PyTorch documentation: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html 

In [0]:
class SimpleImagesDataset(t.utils.data.Dataset):
  def __init__(self, root_path, transforms_x_static = None, transforms_x_dynamic = None, transforms_y_static = None, transforms_y_dynamic = None):
    self.dataset = []
    self.transforms_x = transforms_x_dynamic
    self.transforms_y = transforms_y_dynamic
    for category, img_files in getImagesPaths(root_path):
      # We use only the first 5 images which correspond to images with acetic acid solution
      for i in range(5):
        # The PIL library is one of the most popular solutions for processing images in Python
        img = pil.Image.open(img_files[i])
        # Static transforms are applied on dataset initialization
        if transforms_x_static != None:
          img = transforms_x_static(img)
        if transforms_y_static != None:
          category = transforms_y_static(category)
        self.dataset.append((img, category))    
  
  def __getitem__(self, i):
    x, y = self.dataset[i]
    # Dynamic transformations are applied every time images load
    if self.transforms_x != None:
      x = self.transforms_x(x)
    if self.transforms_y != None:
      y = self.transforms_y(y)
    return x, y

  def __len__(self):
    return len(self.dataset)

## Data augmentation

Neural networks work best when presented to vast amounts of data. For a computer vision task, the dataset we use can be considered small. In order to mitigate reduced training sets we can use image augmentation. This technique creates new training samples from existing ones by applying different transformations to the images, such as deformations, swiping the images vertically or horizontally, zoom, and color variations.

Data transformations for the test and training sets.

In [0]:
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

# input transformation for training set
transforms_train = tv.transforms.Compose([
    # performs rotation between 0 and 45 degrees, scaling between 100% to 200% and shear deformation between 0 and 30 degrees                                    
    tv.transforms.RandomAffine(degrees  = 45, translate = None, scale = (1., 2.), shear = 30),  
    # resizes the image
    tv.transforms.Resize(IMAGE_SIZE),
    # flips the image horizontally with a 1/2 probability
    tv.transforms.RandomHorizontalFlip(),
    # turns the image which is in PIL format to PyTorch tensor
    tv.transforms.ToTensor(),
    # a custom defined transformation for placing the tensor on the GPU, the tensor is on CPU by default
    tv.transforms.Lambda(lambda t: t.cuda()),
    # performs normalization by subtracting the mean and dividing with the standard deviation, both have 3 elements since there are 3 channels for RGB(red, green, blue)
    tv.transforms.Normalize(mean=norm_mean, std=norm_std)    
])

# input transformation for test set
transforms_test = tv.transforms.Compose([
    # resizes the image
    tv.transforms.Resize(IMAGE_SIZE),
    # turns the image which is in PIL format to PyTorch tensor
    tv.transforms.ToTensor(),
    # performs normalization by subtracting the mean and dividing with the standard deviation, both have 3 elements since there are 3 channels for RGB(red, green, blue)
    tv.transforms.Normalize(mean=norm_mean, std=norm_std)    
])

# output transformation of test set, transforms y into a PyTorch tensor of type long and places it on the GPU named 'cuda:0'
y_transform = tv.transforms.Lambda(lambda y: t.tensor(y, dtype=t.long, device = 'cuda:0'))

The dataset and a dataset loader are two different things in PyTorch. The dataset, represent the data itself, while the loader batches the data and performs data augmentation(applies certain transforms for data variety). Furthermore, we can choose to shuffle the data at train time, preventing the network from learning fixed patterns, data shuffling is not needed for network testing(evaluation).

In [0]:
dataset_train = SimpleImagesDataset(TRAIN_PATH, transforms_x_dynamic = transforms_train, transforms_y_dynamic = y_transform)
dataset_test = SimpleImagesDataset(TEST_PATH, transforms_x_static = transforms_test, transforms_x_dynamic = tv.transforms.Lambda(lambda t: t.cuda()), transforms_y_dynamic = y_transform)

# We will need len_tain and len_test later, they express the number of images in the train and test dataset
len_train, len_test = len(dataset_train), len(dataset_test)
print('Training set contains {} samples and test set contains {} samples.'.format(len_train, len_test))

loader_train = t.utils.data.DataLoader(dataset_train, BATCH_SIZE, shuffle = True, num_workers = 0)
loader_test = t.utils.data.DataLoader(dataset_test, BATCH_SIZE, shuffle = False, num_workers = 0)

## Visualize data

Utility function to convert PyTorch tensor to Numpy array. Numpy is probably the most popular Python package for tensor handling and mathematical computing, forming around itself a whole ecosystem, please find our more at the Numpy homepage: https://numpy.org/ 

In [0]:
# places the tensor on cpu, it detaches it from gradient computation and then transforms it to numpy array
def to_numpy(x):
  return x.cpu().detach().numpy()

Functions for plotting one and several images. Please note the usage of matplotlib, one of the most advanced plotting libraries in Python, with endless possibilities.

In [0]:
def plot_one_prediction(x, label):  
  # transform to numpy arrays
  x, label = to_numpy(x), to_numpy(label)
  # matplotlib imshow function expects image format having the channels as a last dimension, 
  # while our network expects the image channels as a first dimension 
  x = np.transpose(x, [1, 2, 0])
  # the image tensors are normalized, we need to undo this transformation
  # x, norm_std and norm_mean are not scalars, are tensors, please note the operators broadcasting to the tensors
  # please find more at: https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html
  x = x * np.array(norm_std) + np.array(norm_mean)
  # set plot title and plot image
  plt.title(label)
  plt.imshow(x)

# compose a single plot out of multiple plots, we will obtain a 4 x 5 images matrix
def plot_predictions(imgs, labels):  
  fig = plt.figure(figsize = (20, 10))
  for i in range(20):
    fig.add_subplot(4, 5, i + 1, xticks = [], yticks = [])
    plot_one_prediction(imgs[i], labels[i])

Load a few images from the training set, so that we can visualize the effects of data augmentation.

In [0]:
# load images and ground truth labels
x, y = next(iter(loader_train))
# plot images
plot_predictions(x, y)

## Define the model

We will be using a ResNet model having 18 layers, pretrained on ImageNet.

Please learn more about this architecture by reading this paper:
https://arxiv.org/abs/1512.03385

Also please check out the torch vision documentation of the models available:
https://pytorch.org/docs/stable/torchvision/models.html

Please try to experiment with other models as well.

In [0]:
def get_resnet_18():
  # download the model
  model = tv.models.resnet18(pretrained = True)
  # the model is pretrained on ImageNet which is a 1000 classification problem, we change the last layer of the network in order to clasify only 4 classes
  model.fc.out_features = 4
  # we place the model on the GPU
  model = model.cuda()
  return model

# t.nn.DataParrallel trains the model on multiple GPU if available, if not it trains on a single one
model = t.nn.DataParallel(get_resnet_18())

## Train & evaluate

This class defines the metrics to be recorded during training.

In [0]:
class Metrics:
  # define the constructor
  def __init__(self):
    self.loss = 0
    self.accuracy = 0

In our case we will train the network in a supervised manner. Supervised training requires the network prediction and the expected result, which is usually called "ground truth". The error is calculated by a loss function which receives as parameters the tensors representing the ground truth and the network prediction. The loss function outputs a scalar value for each training sample, representing the magnitude of the error. In order to make the network learn, we differentiate the loss value with respect to each network parameter (which is a tensor). This implies that the whole network, including the loss function need to be a sequence of differentiable operations. Software implementations make our lives very easy, because they implement the differentiation in an automatic way, by building a computational graph, in which each node represent an operation and each vertex the resulting tensor. Each node (operation) has associated its specific differentiation function. By exploiting the chain rule, each operation (starting from the loss function) is differentiated with respect to its tensors, and those are differentiated recursively with respect to their parent tensors until the differentiation chain reaches the network parameters. Next, each parameter of the network is updated by the negative value of the calculated gradient (derivative of the loss with respect to a network parameter) multiplied by a small number named learning rate. Typically values for the learning rate are between 1e-3 to 1e-5. The learning rate assures proper convergence of the algorithm and its value is crucial in the process of training. After each update of parameters, the network makes the loss value smaller and smaller, thus training of a neural network implies minimizing the loss function with respect to the network parameters. The update of network parameters using the gradients and the learning rate is handled by the optimizer object defined below. The calculation of the network prediction and loss form the forward pass of the network, the calculation of gradients(derivatives of loss with respect to network parameters) forms the backward pass.

In [0]:
loss_fn = t.nn.CrossEntropyLoss()
optimizer = t.optim.Adam(model.parameters(), lr = LEARNING_RATE, weight_decay = 0)

# This function performs a forward pass of the network and records the metrics. 
# If training is ebabled, a backword pass and network parameter updates are also performed.
def run_network(loader, len_dataset, isTrain):  
  # In Python we need to explicitly mark variables which came from outside of a function
  global model, optimizer, loss_fn
  metrics = Metrics()

  # Iterate through the dataset using the data loader
  for x, y in loader:
    # Network forward pass
    y_pred = model.forward(x)
    # Calculate loss value
    loss = loss_fn(y_pred, y)
    
    # Backword pass
    if isTrain:
      assert optimizer != None
      # Gradient backwords propagation 
      loss.backward()
      # Network parameter updates
      optimizer.step()
      # Refresh optimizer state
      optimizer.zero_grad()
    
    # Calculate prediction: y_pred is a batch of arrays of probabilities, 
    # the predicted classes(variable pred) are found by recording the indexes 
    # corresponding to the biggest probabilities in the arrays from y_pred
    y_pred, y = to_numpy(y_pred), to_numpy(y)
    pred = y_pred.argmax(axis = 1)
    
    # The last batch have fewer elements then the rest. 
    # For this reason we weight each metric by the population size of the batch using the variable named 'ratio'
    ratio = len(y) / len_dataset
    metrics.loss += (loss.item() * ratio)
    metrics.accuracy += (sk.metrics.accuracy_score(y, pred) * ratio)
  return metrics

The training happens in a loop, for a certain number of iterations, also named epochs. For each epoch, we go through the whole training dataset and evaluate the model on the test set. We record the metrics for both training set and test set. If the accuracy on the test set is bigger then in the previous rounds we save a checkpoint of the model(we serialize the model). This way, at the end of the training, we are left with a checkpoint of the model that recorded the best accuracy on the test set.

Please check out this PyTorch tutorial for further details about model saving and loading: https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [0]:
# We'll store the metrics per each epoch for plotting
metrics_train_per_epochs, metrics_test_per_epochs = [], []
# If testing on a certain epoch yields better accuracy, then we checkpoint the model
best_acc = 0
    
try:  
  for epoch in range(EPOCHS):    
    # Train
    # We enable the train mode on the model, this activates all dropout and batch normalization layers
    model.train()
    train_metrics = run_network(loader_train, len_train, isTrain = True)
    metrics_train_per_epochs.append(train_metrics)
    
    # Evaluate
    # We enable evaluation mode on the model, this disables all dropout and batch normalization layers
    model.eval()
    # During testing we do not calculate any gradients, nor perform any network parameter updates
    with t.no_grad():
      test_metrics = run_network(loader_test, len_test, isTrain = False)
      metrics_test_per_epochs.append(test_metrics)
      
    # We save a model chekpoint if we find any improvement  
    if test_metrics.accuracy > best_acc:
      best_acc = test_metrics.accuracy
      t.save({'model': model.state_dict()}, 'model checkpoint.tar')

    # Logging  
    print('Epoch {} accuracy {}'.format(epoch + 1, test_metrics.accuracy))

except KeyboardInterrupt as e:
  print('Training interrupted at epoch', epoch)  
print('Ended training')

## Visualize results

Plot a metric per epochs for both train and test data sets.

In [0]:
def plot_train_test(train, test, title, y_title):
    plt.plot(range(len(train)), train, label = 'train')
    plt.plot(range(len(test)), test, label = 'test')
    plt.xlabel('Epochs')
    plt.ylabel(y_title)
    plt.title(title)
    plt.legend()
    plt.show()

Perform actual plotting.

In [0]:
test_accuracies = list(map(lambda m: m.accuracy, metrics_test_per_epochs))
test_loss = list(map(lambda m: m.loss, metrics_test_per_epochs))
train_accuracies = list(map(lambda m: m.accuracy, metrics_train_per_epochs))
train_loss = list(map(lambda m: m.accuracy, metrics_train_per_epochs))

index_max = np.array(test_accuracies).argmax()
print('Best test accuracy :', test_accuracies[index_max])

plot_train_test(train_loss, test_loss, 'Loss', 'Loss')
plot_train_test(train_accuracies, test_accuracies, 'Accuracy', 'Accuracy')

## Open questions

Improving colposcopy through deep learning is an active research area. Here are a few open questions, to spark your curiosity:

**1)** How could we take advantage of the rest of the images, taken through the green lens and with iodine solution ?

**2)** The dataset is unbalanced, how can we counter this phenomenon if we are not able to find additional data ? 

**3)** The model seems to overfit easily while training, how can we counter this ?

**4)** What other architectures and methods can be applied to better model this problem, just a few ideas: ensembles, recurrent networks, object localization, additional image preprocessing, multi-scale convolutions ?  

The questions above cover many possibilities and does not have a precise answer. Deep learning is a field of programming where a problem can be solved in more then one way. Both knowledge and creativity can play an important role. 